In [1]:
import cascade
import pandas as pd
import time
import random

In [2]:
gtfs_path = "/home/chingiz/Rust/py_rust/cascade/cascade-bin/files/Saint_Petersburg"
pbf_path = "/home/chingiz/Rust/osm/roads_SZ.pbf"
departure = 20000
duration = 70000
weekday = "monday"

graph = cascade.create_graph(gtfs_path, pbf_path, departure, duration, weekday)

Filtering left 1941522 rows
Graph creation time: 17.077180208s


In [3]:
point_1 = (30.349061, 59.878163)
point_2 = (30.370268, 59.851074)
list_of_points = [point_1, point_2]
# Define the bounding box
bbox = {
    "min_x": 30.0,
    "max_x": 31.0,
    "min_y": 59.0,
    "max_y": 60.0
}

# Generate 1000 random points within the bounding box
random_points = [
    (
        random.uniform(bbox["min_x"], bbox["max_x"]),
        random.uniform(bbox["min_y"], bbox["max_y"])
    )
    for _ in range(10)
]

list_of_points.extend(random_points)

In [5]:
start = time.perf_counter()
result = cascade.calculate_od_matrix(graph, list_of_points, 43200)
print("Time elapsed", time.perf_counter() - start, len(result))

# crate dataframe from result dict of dict
df: pd.DataFrame = pd.DataFrame.from_dict(result, orient="index").fillna(0)

df_long = df.stack().reset_index()
df_long.columns = ['from_point', 'to_point', 'weight']

Time elapsed 0.8125315149999892 11


In [ ]:
df_long

In [ ]:
mapping = graph.get_mapping()
print(mapping[111].get_geometry())
print(mapping[111].get_id())
print(mapping[111].get_node_type())

print(len(cascade.single_source_shortest_path(graph, 43200, 30.320234, 59.875912)))

print(
    cascade.shortest_path(
        graph=graph,
        dep_time=43200,
        source_x=30.349061,
        source_y=59.878163,
        target_x=30.370268,
        target_y=59.851074,
    )
)